In [ ]:
import torch
import torchvision

# 定義 PyTorch 模型
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()  # 使用半精度運算

# 加載已微調的權重
model.load_state_dict(torch.load('../resnet18_finetuned_fp16_pruned.pth'))

# 模擬輸入張量
data = torch.zeros((1, 3, 224, 134)).cuda().half()

# 將模型導出為 ONNX 格式
torch.onnx.export(
    model,
    data,
    "resnet18_finetuned_fp16_pruned.onnx",
    export_params=True,            # 匯出模型權重
    opset_version=11,               # ONNX 的運算子版本
    do_constant_folding=True,       # 進行常量折疊優化
    input_names=['input'],          # 輸入名稱
    output_names=['output'],        # 輸出名稱
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}  # 動態批量大小
)
print("模型已成功轉換為 ONNX 格式")


In [19]:
import tensorrt as trt
import os

# 設置 TensorRT Logger
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def build_engine_from_onnx(onnx_file_path, engine_file_path, fp16_mode=True):
    # 建立 TensorRT Builder 和 Network
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
        
        # 創建 Builder Config 並設置 max_workspace_size 和 fp16 模式
        config = builder.create_builder_config()
        config.max_workspace_size = 1 << 30  # 設置為 1GB，可以根據需求調整
        if fp16_mode:
            config.set_flag(trt.BuilderFlag.FP16)
        
        # 讀取 ONNX 文件並解析
        if not os.path.exists(onnx_file_path):
            print(f"ONNX 文件 {onnx_file_path} 不存在")
            return None

        with open(onnx_file_path, "rb") as model_file:
            if not parser.parse(model_file.read()):
                print("解析 ONNX 文件失敗")
                for error in range(parser.num_errors):
                    print(parser.get_error(error))
                return None

        # 構建 TensorRT 引擎，並將配置對象傳遞給 build_engine
        engine = builder.build_engine(network, config)
        
        # 將引擎保存到文件
        if engine:
            with open(engine_file_path, "wb") as f:
                f.write(engine.serialize())
            print(f"TensorRT 引擎已成功保存到 {engine_file_path}")
        else:
            print("引擎構建失敗")

        return engine

# 指定 ONNX 文件路徑和生成的 TensorRT 引擎文件路徑
onnx_file_path = "resnet18_finetuned_fp16_pruned.onnx"
engine_file_path = "resnet18_finetuned_fp16_pruned.trt"

# 調用函數進行轉換
build_engine_from_onnx(onnx_file_path, engine_file_path, fp16_mode=True)


AttributeError: 'tensorrt_bindings.tensorrt.IBuilderConfig' object has no attribute 'max_workspace_size'

In [ ]:
import torch
from torch2trt import TRTModule
import os

# 檢查模型檔案是否存在
model_path = 'test_trt.pth'

if not os.path.isfile(model_path):
    print(f"模型檔案未找到: {model_path}")
else:
    print("模型檔案已找到，嘗試加載模型...")

    # 嘗試加載模型
    try:
        model_trt = TRTModule()
        model_trt.load_state_dict(torch.load(model_path))
        print("模型成功加載！")
        
        # 嘗試創建執行上下文
        if model_trt.engine:
            context = model_trt.engine.create_execution_context()
            if context:
                print("成功創建執行上下文！模型可以使用。")
            else:
                print("無法創建執行上下文。請檢查模型的正確性。")
        else:
            print("模型引擎為 None，請檢查模型檔案或重新轉換。")

    except Exception as e:
        print("加載模型時出現錯誤:", e)


輸出差異: [[0.0007324 0.000977 ]]
平均差異: 0.0008545


In [9]:
import time
import torch
import torchvision.transforms as transforms
import cv2
import os
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import torchvision.models as models
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import time
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
# 設定裝置 (若有 GPU 可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import torch
from torch2trt import TRTModule
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('test_trt.pth'))

mean = torch.Tensor([0.4691, 0.4032, 0.4579]).cuda().half()
std = torch.Tensor( [0.1740, 0.1485, 0.1688]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]


# 設定資料夾路徑
image_folder = "../../Train/1600-v4"
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# 設定 widget 以顯示圖片
widget_width = 224
widget_height = 224
image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
display(image_widget)

# 設定裝置 (若有 GPU 可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def bgr8_to_jpeg(image):
    _, jpeg = cv2.imencode('.jpg', image)
    return jpeg.tobytes()

def process_image(image_path):
    # 讀取圖片並進行裁切
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, _ = image.shape
    
    # 裁切圖片的下部 60%
    cropped_image = image[int(height * 0.4):, :, :]  # 保留從 40% 開始到高度底部的部分

    # 調整裁切後的圖片大小為 224x134
    cropped_image = cv2.resize(cropped_image, (224, 134))
    
    # 預處理圖片
    input_tensor = preprocess(cropped_image).unsqueeze(0).to(device)
    
    output = model_trt(input_tensor) 
    xy = model_trt(input_tensor).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    print(x,y/2)
    # 映射預測的 (x, y) 到圖片的像素坐標，假設 x 和 y 是 [0, 1] 範圍內的預測
    x_pixel = int(x * 224 / 2 + 224 / 2)
    y_pixel = int(y * 134 / 2 + 134 / 2)
    
    # 在圖片上繪製預測結果
    display_image = cv2.cvtColor(cropped_image, cv2.COLOR_RGB2BGR)  # 確保格式正確
    cv2.circle(display_image, (x_pixel, y_pixel), 5, (0, 255, 0), -1)  # 綠色點表示預測位置
    
    # 更新 widget 顯示處理後的圖片
    image_widget.value = bgr8_to_jpeg(display_image)

# 對資料夾中的每張圖片進行處理
for image_file in image_files:
    process_image(image_file)
    time.sleep(2)  # 暫停以觀察每張圖片的結果


cuda


C:\Users\Chino\AppData\Local\Temp\ipykernel_53760\3126848648.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_trt.load_state_dict(torch.load('test_trt.pth'))


Image(value=b'', format='jpeg', height='224', width='224')

cuda
0.30810547 -0.0919189453125
0.32666016 -0.0767822265625
0.20080566 -0.0611572265625
0.30126953 -0.063232421875
0.17333984 -0.058837890625
0.359375 -0.064208984375
0.2919922 -0.068359375
0.19335938 -0.0631103515625
0.28027344 -0.0631103515625
0.21923828 -0.06298828125
0.21228027 -0.0523681640625
0.24804688 -0.0589599609375
0.24707031 -0.0545654296875
0.31152344 -0.078857421875
0.23168945 -0.0621337890625
0.38745117 -0.08642578125
0.2529297 -0.0732421875
0.3017578 -0.0546875
0.32763672 -0.05810546875
0.18652344 -0.0472412109375
0.17236328 -0.061767578125
0.31152344 -0.069091796875
0.37939453 -0.072509765625
0.11022949 -0.0667724609375
0.14172363 -0.0640869140625
0.38061523 -0.079833984375
0.20666504 -0.0540771484375
0.21655273 -0.0675048828125
0.24902344 -0.0540771484375
0.15795898 -0.0606689453125
0.3022461 -0.0626220703125
0.30249023 -0.0552978515625
0.32348633 -0.05859375
0.21069336 -0.0697021484375
0.2277832 -0.0479736328125
0.28173828 -0.0914306640625
0.28271484 -0.057250976562

KeyboardInterrupt: 